In [72]:
import os

# Torch
import torch
import torchvision
from torch.utils.data import Dataset
from torchvision import datasets
import torch.optim as optim
from torchvision.transforms import ToTensor, Compose, Normalize
from torchvision.datasets import MNIST
import torch.nn.functional as F
import torch.nn as nn
from PIL import Image

# Use GPU if available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Dataset

In [ ]:
# from roboflow import Roboflow
# rf = Roboflow(api_key="WewILBCNfyY6D0eTbQ2j")
# project = rf.workspace("roboflow-universe-projects").project("fall-detection-ca3o8")
# version = project.version(4)
# dataset = version.download("yolov8")


In [73]:
# define dataset from yolov8 annotations
class CustomDataset(Dataset):
  def __init__(self, dir, transform=None, target_transform=None):
    self.img_dir = f"{dir}/images"
    self.label_dir = f"{dir}/labels"

    # extract file names from the directory, removing the file extension and parent directory
    self.img_names = [os.path.splitext(name)[0] for name in os.listdir(self.img_dir)]

    self.transform = transform
    self.target_transform = target_transform

  def __len__(self):
    return len(self.img_names)

  def __getitem__(self, idx):
    # construct the file paths for the image and label from the directory, add extension
    img_path = os.path.join(self.img_dir, self.img_names[idx] + ".jpg")
    label_path = os.path.join(self.label_dir, self.img_names[idx] + ".txt")
    image = Image.open(img_path)
    labels = [[float(l) for l in line.rstrip('\n').split()] for line in open(label_path)]
    if self.transform:
      image = self.transform(image)  
    if self.target_transform:
      labels = self.target_transform(labels)
    return image, labels

In [76]:
dataset = CustomDataset("Fall-Detection-4/train", transform=ToTensor())

In [42]:
len(dataset)

9438

In [40]:
dataset[0][0].shape

torch.Size([3, 640, 640])

In [78]:
# # compute mean and standard deviation of the dataset
# concat = torch.cat([dataset[i][0] for i in range(4000)], 0).to(torch.float32)
# mean = concat.mean()
# std = concat.std()
# mean, std

(tensor(0.4379), tensor(0.3040))

In [68]:
transform_data = Compose([ToTensor(), Normalize((0.4379,), (0.3040,))])

batch_size = 256
train_dataset = CustomDataset("Fall-Detection-4/train", transform=transform_data)